In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# Distancing Policy

In [2]:
df = pd.read_csv("USstatesCov19distancingpolicy.csv", encoding='latin1')

cols = ['StatePostal', 'StateName', 'StatePolicy','StateWide', 'VaccineExempt', 'PolicyCodingNotes', 
        'PolicySource', 'LastUpdated', 'LastUpdatedNotes', 'ReReviewed']

bar_df = df[(df.StatePolicy == 'BarRestrict') & (df.StateWide == 1)]
bar_df = bar_df[bar_df.DateIssued.values < 20210000]

bar_df.at[(bar_df.DateEnded.values > 20210000) & ~(np.isnan(bar_df.DateEnded.values)), 'DateEnded'] = 20201231
bar_df.at[(bar_df.DateEased.values > 20210000) & ~(np.isnan(bar_df.DateEased.values)), 'DateEased'] = 20201231
bar_df.at[(bar_df.DateExpiry.values > 20210000) & ~(np.isnan(bar_df.DateExpiry.values)), 'DateExpiry'] = 20201231

Enacted_month = []
Enacted_day = []
Ended_month = []
Ended_day = []

for i in range(bar_df.shape[0]):
    row = bar_df.iloc[i]

    if np.isnan(row.DateEnacted):
        Enacted_month.append(int(str(row.DateIssued)[4:6])) 
        Enacted_day.append(int(str(row.DateIssued)[6:8])) 

    else:
        Enacted_month.append(int(str(row.DateEnacted)[4:6])) 
        Enacted_day.append(int(str(row.DateEnacted)[6:8])) 

    if np.isnan(row.DateEnded):
        if np.isnan(row.DateEased):

            if np.isnan(row.DateExpiry):
                Ended_month.append(row.DateExpiry) 
                Ended_day.append(row.DateExpiry) 
            else:
                Ended_month.append(int(str(row.DateExpiry)[4:6])) 
                Ended_day.append(int(str(row.DateExpiry)[6:8])) 

        else:
            Ended_month.append(int(str(row.DateEased)[4:6])) 
            Ended_day.append(int(str(row.DateEased)[6:8])) 

    else:
        Ended_month.append(int(str(row.DateEnded)[4:6])) 
        Ended_day.append(int(str(row.DateEnded)[6:8])) 

bar_df = bar_df[cols]

bar_df['EnactedMonth'] = Enacted_month
bar_df['EnactedDay'] = Enacted_day
bar_df['EndedMonth'] = Ended_month
bar_df['EndedDay'] = Ended_day

bar_df = bar_df.dropna(subset=['EnactedMonth', 'EndedMonth'])

In [3]:
state_labs = np.array(["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"])

li = []

for state in state_labs: 
    ons = np.zeros(12)
    inds = []

    tmpDF = bar_df[bar_df.StatePostal == state]


    for i in range(tmpDF.shape[0]):
        row = tmpDF.iloc[i]
        inds = inds + list(np.arange(row.EnactedMonth - 1, row.EndedMonth))

    inds = np.array(list(set(inds)))
    inds = inds.astype('int')

    ons[inds] = 1

    dates = pd.date_range(start='2020-01', end='2021-01', freq='M')

    li.append(pd.DataFrame({'Date': dates,
                            'State': state,
                            'BarsClosed': ons}))


state_bar_closures = pd.concat(li, axis=0, ignore_index=True)

state_bar_closures.head()

,Date,State,BarsClosed
0,2020-01-31,AL,0.0
1,2020-02-29,AL,0.0
2,2020-03-31,AL,1.0
3,2020-04-30,AL,1.0
4,2020-05-31,AL,1.0


# Covid Cases and Deaths Stats

In [4]:
Covid_df = pd.read_csv('United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv')
Covid_df['submission_date'] =  pd.to_datetime(Covid_df['submission_date'])
Covid_df.index = Covid_df.submission_date.values

li = []

for state in state_labs:
    tmpDF = Covid_df[(Covid_df.state == state) & (Covid_df.submission_date < '2021-01-01')]#.sort_values(by='submission_date')
    tmpDF = tmpDF.groupby(pd.Grouper(freq='M')).sum()
    tmpDF['Date'] = tmpDF.index.values
    tmpDF['State'] = state
    li.append(tmpDF[['Date', 'State', 'new_case', 'new_death']])

Covid_State_df = pd.concat(li, axis=0, ignore_index=True)

In [5]:
Covid_State_df.head()

,Date,State,new_case,new_death
0,2020-01-31,AL,44,0
1,2020-02-29,AL,56,1
2,2020-03-31,AL,3155,48
3,2020-04-30,AL,6805,336
4,2020-05-31,AL,12061,480


# Unemployment Data

In [6]:
U_df = pd.read_csv('Unemployment_Data.csv')
U_df.index = U_df.State.values
U_df = U_df.iloc[:-1, -18:-6]

In [7]:
li = []

for state in state_labs:

    dates = pd.date_range(start='2020-01', end='2021-01', freq='M')

    li.append(pd.DataFrame({'Date': dates,
                            'State': state,
                            'Unemployment': U_df.loc[state].values}))

Unemployment_df = pd.concat(li, axis=0, ignore_index=True)

In [8]:
Unemployment_df.head()

,Date,State,Unemployment
0,2020-01-31,AL,2.7
1,2020-02-29,AL,2.6
2,2020-03-31,AL,2.6
3,2020-04-30,AL,13.2
4,2020-05-31,AL,7.9


# Sandwich them together

In [9]:
print(state_bar_closures.shape)
print(Covid_State_df.shape)
print(Unemployment_df.shape)

(612, 3)
(612, 4)
(612, 3)


In [10]:
State_PD = pd.merge(state_bar_closures, Covid_State_df, on=['Date', 'State'])
State_PD = pd.merge(State_PD, Unemployment_df, on = ['Date', 'State'])

In [11]:
print(State_PD.shape)
State_PD.head()

(612, 6)


,Date,State,BarsClosed,new_case,new_death,Unemployment
0,2020-01-31,AL,0.0,44,0,2.7
1,2020-02-29,AL,0.0,56,1,2.6
2,2020-03-31,AL,1.0,3155,48,2.6
3,2020-04-30,AL,1.0,6805,336,13.2
4,2020-05-31,AL,1.0,12061,480,7.9


In [12]:
State_PD.to_csv('/home/johnattan/Documents/CHIP/Twitter_Analysis/Train_Data/Monthly_Policy_Dem.csv')